In [89]:
%run TF-IDF_rawImplementation.ipynb

In [100]:
#defining the similarity function
from scipy import spatial
import numpy as np
def similarity(u,v):
    return 1 - spatial.distance.cosine(u, v)

In [91]:
def find_top_k(k,doc_sim,index2document):
    related_doc = {}
    for doc,sim in sorted(doc_sim.items(),key = lambda x:x[1], reverse=True):
        related_doc[index2document[doc]] = sim
#         print (index2document[doc])
#         k-=1
#         if k==0:
#             break
    return related_doc

In [92]:
# latent semantic indexing (LSI)
from numpy import linalg

'''
    query:indices of words appear in matrix
    k:dimention of latent space
'''

def latent_semantic_indexing(q, tf_idf, k, topk, word2index, index2document):
    # preprocssing the query vector
    query = []
    q_v = np.zeros(len(word2index))
    for w in q.split():
        try:
            q_v[word2index[w]]+=1
            query.append(word2index[w])
        except:
            print('keywords not found')
    print(query)        
    doc_similarity = {}
    u,s,v = linalg.svd(tf_idf)
    u = u[:,:k]
    s_ = np.zeros((k,k))
    for i in range(k):
        s_[i,i] = s[i]
    v = v[:k,:]
    q_v = np.zeros(len(word2index))
    for q in query:
        q_v[q]+=1
    q_v = q_v.reshape(1,-1)
    q_v = np.matmul(q_v,u)
    s_ = linalg.inv(s_)
    q_v = np.matmul(q_v,s_)
    
    for i in range(v.shape[1]):
        sim = similarity(q_v,v[:,i])
        doc_similarity[i] = sim 
    print(doc_similarity)
    related_doc = find_top_k(topk, doc_similarity,index2document)
    print(related_doc)
    return related_doc

In [43]:
# %%time
# raw_doc_path = '/home/bit/ma0/LabShare/data/three-companies/json'
# tf_idf, word2index, index2document, inv_doc_freq = tf_idf_processing(raw_doc_path)

In [93]:
%%time
data_path = '/home/bit/ma0/LabShare/data/chui_ma/Presentation_data/'
tf_idf, word2index, index2document, inv_doc_freq = tf_idf_PrerocessedDoc(data_path)

CPU times: user 612 ms, sys: 24 ms, total: 636 ms
Wall time: 383 ms


In [46]:
# q = 'Risiko Kapitalanlage Höhe'
# latent_semantic_indexing(q, tf_idf, 3, 5, word2index, index2document)

In [94]:
%%time
u,s,v = linalg.svd(tf_idf)

CPU times: user 1min 20s, sys: 6.4 s, total: 1min 27s
Wall time: 17.4 s


In [165]:
import matplotlib.pyplot as plt
from matplotlib import colors
def getRelatedDocuments(q, u, s, v, word2index, index2document):
    query = []
    # q = 'kapitalanlage risiko höhe '
#     q = 'trägerraketen flugzeug'
    q_v = np.zeros(len(word2index))
    for w in q.split():
        try:
            q_v[word2index[w]]+=1
            query.append(word2index[w])
        except:
            print('keywords "{0}" not found'.format(w))
#     print(query)        
    doc_similarity = {}
    k = 3
    topk = 10
    u = u[:,:k]
    s_ = np.zeros((k,k))
    for i in range(k):
        s_[i,i] = s[i]
    v = v[:k,:]
    q_v = np.zeros(len(word2index))
    for q in query:
        q_v[q]+=1
    q_v = q_v.reshape(1,-1)
    q_v = np.matmul(q_v,u)
    s_ = linalg.inv(s_)
    q_v = np.matmul(q_v,s_)

    for i in range(v.shape[1]):
        sim = similarity(q_v,v[:,i])
        doc_similarity[i] = sim 
    risiko_related_doc = find_top_k(topk, doc_similarity,index2document)
    

    def background_gradient(s, m, M, cmap='PuBu', low=0, high=0):
        rng = M - m
        norm = colors.Normalize(m - (rng * low),
                                M + (rng * high))
        normed = norm(s.values)
        c = [colors.rgb2hex(x) for x in plt.cm.get_cmap(cmap)(normed)]
        return ['background-color: %s' % color for color in c]
    df = pd.DataFrame(risiko_related_doc, index=['relevance score'])
    # risiko_related_doc
    df = df.transpose()
    return df.style.apply(background_gradient,
                   cmap='PuBu',
                   m=df.min().min(),
                   M=df.max().max(),
                   low=0,
                   high=0.2)

In [166]:
# q = ' umsatz teigerung'
# q = ' steigerung'

q = 'ich möchte ein schuh oder eine socke kaufen'
getRelatedDocuments(q, u, s, v, word2index, index2document)

keywords "ich" not found
keywords "ein" not found
keywords "oder" not found
keywords "eine" not found


,relevance score
PUMA-2015,0.999993
PUMA-2016,0.999962
Adidas-2016,0.917364
Adidas-2015,0.901589
Bayer-2015,0.460084
Bayer-2016,0.441746
Deutsche_Post-2015,0.316022
Deutsche_Post-2016,0.311603
BMW-2015,0.299824
BMW-2016,0.278186


In [86]:
query = []
# q = 'kapitalanlage risiko höhe '
q = 'trägerraketen flugzeug'
q_v = np.zeros(len(word2index))
for w in q.split():
    try:
        q_v[word2index[w]]+=1
        query.append(word2index[w])
    except:
        print('keywords not found')
print(query)        
doc_similarity = {}
k = 3
topk = 10
u = u[:,:k]
s_ = np.zeros((k,k))
for i in range(k):
    s_[i,i] = s[i]
v = v[:k,:]
q_v = np.zeros(len(word2index))
for q in query:
    q_v[q]+=1
q_v = q_v.reshape(1,-1)
q_v = np.matmul(q_v,u)
s_ = linalg.inv(s_)
q_v = np.matmul(q_v,s_)

for i in range(v.shape[1]):
    sim = similarity(q_v,v[:,i])
    doc_similarity[i] = sim 
risiko_related_doc = find_top_k(topk, doc_similarity,index2document)

[25378, 9067]


In [87]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors

def background_gradient(s, m, M, cmap='PuBu', low=0, high=0):
    rng = M - m
    norm = colors.Normalize(m - (rng * low),
                            M + (rng * high))
    normed = norm(s.values)
    c = [colors.rgb2hex(x) for x in plt.cm.get_cmap(cmap)(normed)]
    return ['background-color: %s' % color for color in c]



In [88]:
import pandas as pd
df = pd.DataFrame(risiko_related_doc, index=['relevance score'])
# risiko_related_doc
df = df.transpose()
df.style.apply(background_gradient,
               cmap='PuBu',
               m=df.min().min(),
               M=df.max().max(),
               low=0,
               high=0.2)

,relevance score
Airbus-AnnualReport-2012.txt,0.99974
Airbus-AnnualReport-2013.txt,0.99969
Airbus-AnnualReport-2010.txt,0.999543
Airbus-AnnualReport-2011.txt,0.999363
Airbus-AnnualReport-2014.txt,0.99898
Deutsche_Post-AnnualReport-2012.txt,0.197258
Deutsche_Post-AnnualReport-2011.txt,0.189591
Deutsche_Post-AnnualReport-2010.txt,0.169077
Bayer-AnnualReport-2010.txt,0.158714
Bayer-AnnualReport-2011.txt,0.155929
